In [36]:
# Import af nødvendige packages
import quandl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [37]:
""" SETTINGS """
source = 'csv' # quandl eller csv
data_interval = 'monthly' # daily, monthy eller yearly - Hvis Quandl = 'daily'
selected = ['NoDur', 'Durbl', 'Manuf', 'Enrgy', 'HiTec', 'Telcm', 'Shops', 'Hlth', 'Utils', 'Other', 'Mkt'] # Angiv ønskede aktiver fra Quandl eller kolonner fra CSV-ark

# Hvis CVS:
file_name = '10 Industry Portfolios - Average Value Weighted Returns.CSV' # navn og file-exstention på data

# Hvis Quandl
date_range = ['2014-1-1', '2016-12-31'] # dato fra og med - dato til og med, format YYYY-MM-DD

In [38]:
# Træk af data fra Quandl
if source == 'quandl':
    quandl.ApiConfig.api_key = "yTPaspmH6wqs9rAdSdmk"
    data = quandl.get_table('WIKI/PRICES', ticker = selected, qopts = {'columns': ['date', 'ticker', 'adj_close']},
                            date = {'gte': date_range[0], 'lte': date_range[1]}, paginate=True)
    clean = data.set_index('date')
    table = clean.pivot(columns='ticker')
    returns_data = table.pct_change()
    
# Træk af data fra CSV-fil (allerede procent-vis ændring)
elif source == 'csv':
    rf = pd.read_csv(file_name, delimiter = ',', header = 0, usecols = ['RF3'])
    data = pd.read_csv(file_name, delimiter = ',', header = 0, index_col = 'date', usecols = ['date'] + selected)
    returns_data = np.subtract(data,rf)
    
actives = list(returns_data.columns.values)
returns_data.head() # Eksempel på den procentvise ændring

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other,Mkt
date,,,,,,,,,,,
196307,-0.740000,-0.490000,-1.680000,2.020000,-0.960000,-0.500000,-1.300000,0.290000,0.530000,-1.880000,-0.390000
196308,4.610000,6.290000,5.940000,3.670000,4.880000,4.030000,6.140000,9.300000,3.940000,5.230000,5.060000
196309,-1.943333,-0.503333,-1.023333,-3.903333,-0.133333,2.096667,0.696667,-4.323333,-2.763333,-3.423333,-1.563333
196310,2.390000,9.450000,2.310000,-0.590000,8.020000,3.130000,0.210000,3.110000,-0.940000,1.110000,2.550000
196311,-1.406667,-5.116667,0.023333,-1.426667,-0.566667,3.883333,-1.506667,-1.926667,-1.296667,-0.046667,-0.856667


In [39]:
port_returns = []
port_volatility = []
sharpe_ratio = []
stock_weights = []
M = 120
N = len(selected)

In [40]:
for month in range(0, len(returns_data) - M):
    # Danner nyt data interval, returns og cov-matrix for hvert interval
    new_returns_data = returns_data[month:month + M]
    returns_monthly = new_returns_data
    returns_mean = returns_monthly.mean()
    cov_matrix = returns_monthly.cov()

    tmin = np.dot(np.linalg.inv(cov_matrix), np.ones(11))
    nmin = np.dot(np.ones(11), np.dot(np.linalg.inv(cov_matrix), np.ones(11)))
    wmin = tmin / nmin
    mumin = np.dot(wmin.T, returns_mean)
    muny = np.subtract(returns_mean, np.ones(11)*mumin)
    element1 = np.dot(np.transpose(muny), np.dot(np.linalg.inv(cov_matrix), muny))
    Phi = (N + 2) / ((N + 2) + M * element1)
    mubs = (1 - Phi) * returns_mean + Phi * mumin
    tbs = np.dot(np.linalg.inv(cov_matrix), mubs)
    nbs = np.dot(np.ones(11), np.dot(np.linalg.inv(cov_matrix), mubs))
    weights = tbs / nbs

    # udregner afkast for måned T + 1 med de udregnede optimale vægte
    returns = np.dot(weights, returns_data[month + M:month + M + 1].mean())
    volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe = returns / volatility
    
    # indsætter overstående udregninger i vektorerne
    sharpe_ratio.append(sharpe)
    port_returns.append(returns)
    port_volatility.append(volatility)
    stock_weights.append(weights)

In [41]:
# definerer portfolier som en sammensætning af overstående udregninger
portfolio = {'Returns': port_returns,
             'Volatility': port_volatility,
             'Sharpe Ratio': sharpe_ratio}

In [42]:
# definerer de enkelte vægte af de valgte aktiver, HINT: de er 0.33 alle sammen da det er 1/N
for i in range(len((actives))):
    symbol = actives[i]
    portfolio[symbol+' Weight'] = [Weight[i] for Weight in stock_weights]

In [43]:
# sætter vores portføljer som dataframe i pandas
df = pd.DataFrame(portfolio)
column_order = ['Returns', 'Volatility', 'Sharpe Ratio'] + [symbol+' Weight' for symbol in actives]
df = df[column_order]

df.head() # Porteføljer ud fra 60 måneders interval
          # ie. portfølje '0' har return, vol og sharpe ud fra måned 1 - 60
          # portfølje '1' har således overstående ud fra måned 2 - 61 ovs.

,Returns,Volatility,Sharpe Ratio,NoDur Weight,Durbl Weight,Manuf Weight,Enrgy Weight,HiTec Weight,Telcm Weight,Shops Weight,Hlth Weight,Utils Weight,Other Weight,Mkt Weight
0,-10.689757,7.118219,-1.501746,3.144875,2.789075,8.501586,5.387496,3.315805,2.392894,3.188679,3.559306,2.230706,2.352535,-35.862958
1,-14.277675,7.565915,-1.887105,3.147069,2.772430,8.749151,5.415718,3.205931,2.415381,3.498939,3.731907,2.029774,2.501709,-36.468008
2,4.746920,8.637165,0.549592,3.540347,3.146057,10.796814,6.340838,3.759484,2.683295,4.125399,4.091970,2.446319,3.022988,-42.953512
3,1.511400,7.914350,0.190970,3.353801,2.930281,9.772423,5.909404,3.343668,2.413648,3.713922,3.807861,2.320655,2.899343,-39.465007
4,-1.438952,9.172307,-0.156880,3.301092,2.841682,11.356792,6.791728,3.590616,2.437333,4.432907,4.256213,2.590996,3.298891,-43.898251


In [44]:
df['Returns'].mean() / df['Returns'].std()

0.07239856324417297

In [45]:
df['Sharpe Ratio'].mean()

0.07099463679367413